Data Ingestion

In [1]:
##DATA INGESTION
from langchain_community.document_loaders import TextLoader
loader = TextLoader("sample.txt")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'sample.txt'}, page_content='Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It has various subfields such as machine learning, natural language processing, robotics, and computer vision.\n\nMachine learning (ML) is a subset of AI that enables systems to learn from data and improve over time without being explicitly programmed. Supervised, unsupervised, and reinforcement learning are the three main types of ML.\n\nNatural Language Processing (NLP) is the branch of AI that deals with the interaction between computers and humans through natural language. NLP enables machines to understand, interpret, and generate human language.\n\nOne popular application of NLP is the chatbot, which can simulate conversation with users. These bots are commonly used in customer service, virtual assistants, and help desk systems.\n\nAnother key technology in AI is computer vision, whic

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 6


In [7]:
## web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
##load,chunk and index the content of the website

loader = WebBaseLoader( web_paths=["https://www.geeksforgeeks.org/java/object-oriented-programming-oops-concept-in-java/"],
                      bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                        class_=("article-title","text")
                      )),)

text_documents = loader.load()
text_documents

[Document(metadata={'source': 'https://www.geeksforgeeks.org/java/object-oriented-programming-oops-concept-in-java/'}, page_content='\nJava OOP(Object Oriented Programming) Concepts\n\n\nBefore Object-Oriented Programming (OOPs), most programs used a procedural approach, where the focus was on writing step-by-step functions. This made it harder to manage and reuse code in large applications.To overcome these limitations, Object-Oriented Programming was introduced. Java is built around OOPs, which helps in organizing code using classes and objects.Key Features of OOP in Java:Structures code into logical units (classes and objects)Keeps related data and methods together (encapsulation)Makes code modular, reusable and scalablePrevents unauthorized access to dataFollows the DRY (Don’t Repeat Yourself) principleCharacteristics of an OOP(Object Oriented Programming) The diagram below demonstrates the Java OOPs ConceptsOOP\'s concept1. ClassA Class is a user-defined blueprint or prototype fro

In [10]:
##pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("sample_pdf.pdf")

docs = loader.load()
docs



[Document(metadata={'producer': 'GPL Ghostscript 9.56.1', 'creator': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creationdate': "D:20250821052712Z00'00'", 'moddate': "D:20250821052712Z00'00'", 'source': 'sample_pdf.pdf', 'total_pages': 10, 'page': 0, 'page_label': '1'}, page_content='AI-Powered Database \nCapacity Planning & \nPerformance Forecasting\nEmpowering Database Administrators and SREs with proactive \ninsights for optimal performance.'),
 Document(metadata={'producer': 'GPL Ghostscript 9.56.1', 'creator': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creationdate': "D:20250821052712Z00'00'", 'moddate': "D:20250821052712Z00'00'", 'source': 'sample_pdf.pdf', 'total_pages': 10, 'page': 1, 'page_label': '2'}, page_content='Chapter 1\nThe Hidden Challenges of \nDatabase Growth\nDatabases are the heart of modern applications, but their continuous growth can lead to subtle, yet critical, \nperformance problems. What starts as a small issue can quickly escalate into widesprea

Transforming(Converting data in smaller chunks)

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'GPL Ghostscript 9.56.1', 'creator': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creationdate': "D:20250821052712Z00'00'", 'moddate': "D:20250821052712Z00'00'", 'source': 'sample_pdf.pdf', 'total_pages': 10, 'page': 0, 'page_label': '1'}, page_content='AI-Powered Database \nCapacity Planning & \nPerformance Forecasting\nEmpowering Database Administrators and SREs with proactive \ninsights for optimal performance.'),
 Document(metadata={'producer': 'GPL Ghostscript 9.56.1', 'creator': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creationdate': "D:20250821052712Z00'00'", 'moddate': "D:20250821052712Z00'00'", 'source': 'sample_pdf.pdf', 'total_pages': 10, 'page': 1, 'page_label': '2'}, page_content='Chapter 1\nThe Hidden Challenges of \nDatabase Growth\nDatabases are the heart of modern applications, but their continuous growth can lead to subtle, yet critical, \nperformance problems. What starts as a small issue can quickly escalate into widesprea

In [12]:
##Vector Embedding and Vector Store
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents[:20],OpenAIEmbeddings())

C:\Users\user\AppData\Local\Temp\ipykernel_18904\3880645925.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = Chroma.from_documents(documents[:20],OpenAIEmbeddings())


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Now we can query this Vector Database

In [13]:
## Vector Database
query = "What is polymorphism?"
result = db.similarity_search(query, k=1)
print(result[0].page_content)

NameError: name 'db' is not defined

In [ ]:
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
db1 = FAISS.from_documents(documents[:20],OpenAIEmbeddings())
query = "What is polymorphism?"
result = db1.similarity_search(query, k=1)
print(result[0].page_content)


